In [22]:
#https://github.com/kmsravindra/ML-AI-experiments/blob/master/AI/Neural%20Machine%20Translation/Neural%20machine%20translation%20-%20Encoder-Decoder%20seq2seq%20model.ipynb
import os
"""
os.environ["CUDA_VISIBLE_DEVICES"]="0"
import tensorflow as tf
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)
from keras import backend as K
K.set_session(sess)"""

from keras.models import Model, load_model
from keras.layers import Input, LSTM, Dense, Bidirectional, Concatenate
from keras import optimizers
import numpy as np
import pandas as pd
from keras.callbacks import ModelCheckpoint
from sklearn.metrics import mean_squared_error
from time import gmtime, strftime

Using TensorFlow backend.


In [23]:
session_name = "Session_normal_dataset"
timestamp = strftime("%d.%d.%y_%H:%M", gmtime())
folder_name = session_name+"_"+timestamp

folder_name = "trained_models/"+session_name+"_"+timestamp
weights_folder_name = folder_name + "/weights"
 
try:
    # Create target Directory
    os.mkdir(folder_name)
    print("Directory " , folder_name ,  " Created ") 
except FileExistsError:
    print("Directory " , folder_name ,  " already exists")
    
try:
    # Create target Directory
    os.mkdir(weights_folder_name)
    print("Directory " , weights_folder_name ,  " Created ") 
except FileExistsError:
    print("Directory " , weights_folder_name ,  " already exists")

Directory  trained_models/Session_normal_dataset_09.09.18_12:57  Created 
Directory  trained_models/Session_normal_dataset_09.09.18_12:57/weights  Created 


In [24]:
batch_size=128
epochs=1

In [25]:
# Load dataset and create pairs of inputs and targets

with np.load('split_inputs.npz') as split_inputs:
    inputs = split_inputs['train']
    
with np.load('split_targets.npz') as split_targets:
    targets = split_targets['train']

print("Number of training inputs",len(inputs))
print("Number of training targets",len(targets))

# Print examples of inputs and targets
example_in = inputs[0]
example_target = targets[0]
print("Input example:", example_in)
print("Target example:", example_target)

Number of training inputs 117500
Number of training targets 117500
Input example: [ 65 129 129 129  72 129 129 129  69 129 129 129  65 129 129 129  72 129
 129 129  74 129 129 129  74 129 129 129  72 129 129 129  72 129 129 129
  74 129 129 129  76 129 129 129  77 129 129 129  76 129 129 129  74 129
 129 129  74 129 129 129  72 129 129 129]
Target example: [ 60 129 129 129  60 129 129 129  60 129 129 129  62 129 129 129  65 129
 129 129  70 129 129 129  70 129 129 129  69 129 129 129  69 129 129 129
  70 129 129 129  70 129 129 129  72 129  74 129  72 129  67 129  69 129
 129 129  67 129 129 129  67 129 129 129]


In [26]:
# Load test set
with np.load('split_test_inputs.npz') as test_inputs:
    test_soprano_tenor = test_inputs['test']
with np.load('split_test_targets.npz') as test_targets:
    test_alto_bass = test_targets['test']

In [27]:
# Make set of unique notes
split_inputs_uniq = np.unique(inputs)
split_targets_uniq = np.unique(targets)
test_soprano_tenor_uniq = np.unique(test_soprano_tenor)
test_alto_bass_uniq = np.unique(test_alto_bass)
input_set = set()
target_set = set()
for note in split_inputs_uniq:
    input_set.add(note)
    target_set.add(note)
for note in split_targets_uniq:
    input_set.add(note)
    target_set.add(note)
for note in test_soprano_tenor_uniq:
    input_set.add(note)
    target_set.add(note)
for note in test_alto_bass_uniq:
    input_set.add(note)
    target_set.add(note)
target_set.add(200)
target_set.add(201)

In [28]:
input_sequences = list(inputs)
target_sequences = []
#input_set = set()
#target_set = set()
num_samples = len(inputs)
    
for i in range(num_samples):    
    target_with_tokens = [200] + list(targets[i]) + [201] # use 200 as start token and 201 as end token
    target_sequences.append(target_with_tokens)
    
    """# Add unique notes to set of notes
    for item in input_sequences[i]:
        if (item not in input_set):
            input_set.add(item)
    
    for item in target_with_tokens:
        if (item not in target_set):
            target_set.add(item)"""
            
input_set = sorted(list(input_set))
target_set = sorted(list(target_set))
print("Set of input notes:\n", input_set)
print("Set of target notes:\n", target_set)

Set of input notes:
 [36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 128, 129]
Set of target notes:
 [36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 128, 129, 200, 201]


In [29]:
# dictionary to index each input note - key is index and value is the note
input_index_to_note_dict = {}

# dictionary to get note given its index - key is the note and value is the index
input_note_to_index_dict = {}

for k, v in enumerate(input_set):
    input_index_to_note_dict[k] = v
    input_note_to_index_dict[v] = k

In [30]:
# dictionary to index each target note - key is index and value is the note
target_index_to_note_dict = {}

# dictionary to get note given its index - key is the note and value is the index
target_note_to_index_dict = {}

for k, v in enumerate(target_set):
    target_index_to_note_dict[k] = v
    target_note_to_index_dict[v] = k

In [31]:
max_len_inputs = max([len(seq) for seq in input_sequences])
max_len_targets = max([len(line) for line in target_sequences])
print(max_len_inputs)
print(max_len_targets)

64
66


In [32]:
tokenized_input_sequences = np.zeros(shape=(num_samples,max_len_inputs,len(input_set)), dtype='float32')
tokenized_target_sequences = np.zeros(shape=(num_samples,max_len_targets,len(target_set)), dtype='float32')
target_data = np.zeros((num_samples, max_len_targets, len(target_set)),dtype='float32')

In [33]:
for i in range(num_samples):
    for k, note_value in enumerate(input_sequences[i]):
        tokenized_input_sequences[i,k,input_note_to_index_dict[note_value]] = 1
    
    for k, note_value in enumerate(target_sequences[i]):
        tokenized_target_sequences[i,k,target_note_to_index_dict[note_value]] = 1
        
        # decoder_target_data will be ahead by one timestep and will not include the start character.
        if k > 0:
            target_data[i,k-1, target_note_to_index_dict[note_value]] = 1

In [34]:
# Encoder model

encoder_input = Input(shape=(None,len(input_set)))
encoder_LSTM = Bidirectional(LSTM(256,return_state = True))
encoder_outputs, encoder_h_forward, encoder_c_forward, encoder_h_back, encoder_c_back = encoder_LSTM (encoder_input)

# Concatenate forward states and backward states
encoder_h = Concatenate()([encoder_h_forward, encoder_h_back])
encoder_c = Concatenate()([encoder_c_forward, encoder_c_back])
#encoder_h = concatenate([encoder_h_forward, encoder_h_back])
#encoder_c = concatenate([encoder_c_forward, encoder_c_back])
encoder_states = [encoder_h, encoder_c]

In [35]:
print(encoder_h_forward)
print(encoder_h_back)

print(encoder_h)
print(encoder_c)

Tensor("bidirectional_1/while/Exit_2:0", shape=(?, 256), dtype=float32)
Tensor("bidirectional_1/while_1/Exit_2:0", shape=(?, 256), dtype=float32)
Tensor("concatenate_1/concat:0", shape=(?, 512), dtype=float32)
Tensor("concatenate_2/concat:0", shape=(?, 512), dtype=float32)


In [36]:
# Decoder model

decoder_input = Input(shape=(None,len(target_set)))

# 512 because of concatenated forward states and backward states
decoder_LSTM = LSTM(256*2,return_sequences=True, return_state = True)
decoder_out, _ , _ = decoder_LSTM(decoder_input, initial_state=encoder_states)
decoder_dense = Dense(len(target_set),activation='softmax')
decoder_out = decoder_dense (decoder_out)

In [41]:
filepath=weights_folder_name+"/{epoch:02d}-{mean_squared_error:.2f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, mode='max')
callbacks_list = [checkpoint]

In [42]:
model = Model(inputs=[encoder_input, decoder_input],outputs=[decoder_out])

# Run training
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(x=[tokenized_input_sequences,tokenized_target_sequences], 
          y=target_data,
          batch_size=batch_size,
          epochs=epochs,
          validation_split=0.2,callbacks=callbacks_list)


Train on 94000 samples, validate on 23500 samples
Epoch 1/1
10112/94000 [==>...........................] - ETA: 22:41 - loss: 1.4209 - acc: 0.6991

KeyboardInterrupt: 

In [43]:
# Inference models for testing

# Encoder inference model
encoder_model_inf = Model(encoder_input, encoder_states)

# Decoder inference model
decoder_state_input_h = Input(shape=(256*2,))
decoder_state_input_c = Input(shape=(256*2,))
decoder_input_states = [decoder_state_input_h, decoder_state_input_c]

decoder_out, decoder_h, decoder_c = decoder_LSTM(decoder_input, 
                                                 initial_state=decoder_input_states)

decoder_states = [decoder_h , decoder_c]

decoder_out = decoder_dense(decoder_out)

decoder_model_inf = Model(inputs=[decoder_input] + decoder_input_states,
                          outputs=[decoder_out] + decoder_states )

In [56]:
def decode_seq(inp_seq, input_length):
    
    # Initial states value is coming from the encoder 
    states_val = encoder_model_inf.predict(inp_seq)
   
    target_seq = np.zeros((1, 1, len(target_set)))
    target_seq[0, 0, target_note_to_index_dict[200]] = 1

    decoded_sequence = []
    stop_condition = False
    input_seq_length = len(inp_seq)
    
    while not stop_condition:       
        decoder_out, decoder_h, decoder_c = decoder_model_inf.predict(x=[target_seq] + states_val)      
        max_val_index = np.argmax(decoder_out[0,-1,:])
        sampled_note = target_index_to_note_dict[max_val_index]
        decoded_sequence.append(sampled_note)
        
        if ((sampled_note == 201) or (len(decoded_sequence) == input_length)):
        #if ((sampled_note == 201) or (len(decoded_sequence) > max_len_targets)):
            stop_condition = True
        
        target_seq = np.zeros((1, 1, len(target_set)))
        target_seq[0, 0, max_val_index] = 1
        
        states_val = [decoder_h, decoder_c]   
    return np.array(decoded_sequence)

In [57]:
# Sample with soprano and tenor from test set

with np.load('test_soprano_tenor.npz') as test_targets:
    test_soprano_tenor = test_targets['test']
    

max_len_test_soprano_tenor = max([len(seq) for seq in test_soprano_tenor])
tokenized_inputs_sop_ten = np.zeros(shape=(len(test_soprano_tenor),max_len_test_soprano_tenor,len(input_set)), dtype='float32')  
    
for i in range(len(test_soprano_tenor)):
    for k, note_value in enumerate(test_soprano_tenor[i]):
        tokenized_inputs_sop_ten[i,k,input_note_to_index_dict[note_value]] = 1 


# Save all samples as file.
# Input sequences on even index and decoded sequences on odd index

folder_name = "samples"

samples = []
for seq_index, seq in enumerate(test_soprano_tenor):
    if seq_index % 10 == 0:
        print(seq_index)
    inp_seq = tokenized_inputs_sop_ten[seq_index:seq_index+1]
    input_seq_length = len(seq)
    decoded_sequence = decode_seq(inp_seq, input_seq_length)
    samples.append(test_soprano_tenor[seq_index])
    samples.append(decoded_sequence)
samples = np.array(samples)
np.savez(folder_name+'/sampled_soprano_tenor.npz', samples=samples)

80
0
10


In [ ]:
# Sample with alto and bass from test set

with np.load('test_alto_bass.npz') as test_targets:
    test_alto_bass = test_targets['test']

max_len_test_alto_bass = max([len(seq) for seq in test_alto_bass])
tokenized_inputs_alt_bas = np.zeros(shape=(len(test_alto_bass),max_len_test_alto_bass,len(input_set)), dtype='float32')  
    
for i in range(len(test_soprano_tenor)):
    for k, note_value in enumerate(test_alto_bass[i]):
        tokenized_inputs_alt_bas[i,k,input_note_to_index_dict[note_value]] = 1 


# Save all samples as file.
# Input sequences on even index and decoded sequences on odd index

folder_name = "samples"

samples = []
for seq_index, seq in enumerate(test_alto_bass):
    if seq_index % 10 == 0:
        print(seq_index)
    inp_seq = tokenized_inputs_alt_bas[seq_index:seq_index+1]
    input_seq_length = len(seq)
    decoded_sequence = decode_seq(inp_seq, input_seq_length)
    samples.append(test_alto_bass[seq_index])
    samples.append(decoded_sequence)
samples = np.array(samples)
np.savez(folder_name+'/sampled_alto_bass.npz', samples=samples)

165
